In [1]:
#!fsharp
// installation of paket integration (did not find out to do it properly)
let installDesignTime () = 
    let home = 
        match System.Environment.OSVersion.Platform with
        | PlatformID.Unix | PlatformID.MacOSX -> Environment.GetEnvironmentVariable("HOME")
        | _ -> Environment.ExpandEnvironmentVariables("%HOMEDRIVE%%HOMEPATH%")
    let target = System.IO.Path.Combine(home,@".nuget\packages\microsoft.dotnet-interactive\1.0.156101\tools\netcoreapp3.1\any\FSharp.DependencyManager.Paket.dll")
    System.IO.File.Copy(@"packages\build\FSharp.DependencyManager.Paket\lib\netstandard2.0\FSharp.DependencyManager.Paket.dll", target)

In [1]:
#!fsharp
installDesignTime()

In [1]:
#!fsharp
#r "paket: nuget FSharp.Data.Adaptive"
#r "paket: nuget Aardvark.Base"
#r "paket: nuget Aardvark.Base.Incremental"
#r "paket: nuget Aardvark.Base.FSharp"
#r "paket: nuget Aardvark.Base.Rendering"
#r "paket: nuget Aardvark.Application.Slim"
#r "paket: nuget Aardvark.Application.Slim.GL"
#r "paket: nuget Aardvark.Application.Slim.Vulkan"
#r "paket: nuget Aardvark.SceneGraph"
#r "paket: nuget Aardvark.SceneGraph.IO"
#r "paket: nuget Aardvark.Service"
#r "paket: nuget Aardvark.UI"
#r "paket: nuget Aardvark.UI.Primitives"

ok paket


err paket


In [1]:
#!fsharp
// can be removed using project reference in the future
#r @"bin\Debug\netcoreapp3.1\NotebookApp.dll"

In [1]:
#!fsharp
open Aardvark.Notebooks
Notebooks.init()

In [1]:
#!fsharp
let show (v : Aardvark.Notebooks.NotebookUtilities.NotebookView) = 
    let html = sprintf "<iframe src='%s' width='800' height='400'></frame>" v.Url
    Html.ToHtmlContent html

In [1]:
#!fsharp
open NotebookApp

let app = new Aardvark.Application.Slim.VulkanApplication()

In [1]:
#!fsharp
let napp = NotebookApp(app.Runtime, App.app)
napp.AddView(App.view) |> show


In [1]:
#!fsharp
open Aardvark.Base
open Aardvark.UI
open Aardvark.UI.Primitives
open FSharp.Data.Adaptive
open NotebookApp.Model
open Aardvark.Base.Rendering

In [1]:
#!fsharp
let view (m : AdaptiveModel) =

    let frustum = 
        Frustum.perspective 60.0 0.1 100.0 1.0 
            |> AVal.constant

    let sg =
        m.currentModel |> AVal.map (fun v ->
            match v with
                | Box -> Sg.box (AVal.constant C4b.Red) (AVal.constant (Box3d(-V3d.III, V3d.III)))
                | Sphere -> Sg.sphere 5 (AVal.constant C4b.Green) (AVal.constant 1.0)
        )
        |> Sg.dynamic
        |> Sg.shader {
            do! DefaultSurfaces.trafo
            do! DefaultSurfaces.simpleLighting
        }

    let att =
        [
            style "position: fixed; left: 0; top: 0; width: 100%; height: 100%"
        ]

    body [] [
        FreeFlyController.controlledControl m.cameraState CameraMessage frustum (AttributeMap.ofList att) sg

        div [style "position: fixed; left: 20px; top: 20px"] [
            button [onClick (fun _ -> ToggleModel)] [text "Toggle Model"]

            Incremental.text (AVal.map string m.cnt)
        ]
    ]

napp.AddView(view) |> show